task name: MapKeyToValueMultiColumn
task description: Implement the MapKeyToValueMultiColumn example using ML.NET

In [ ]:
#r "nuget:Microsoft.ML,1.6.0"

In [ ]:
using System;
using System.Collections.Generic;
using Microsoft.ML;
using Microsoft.ML.Data;

In [ ]:
var mlContext = new MLContext(seed: 0);

In [ ]:
private class DataPoint
{
    public string Label { get; set; }
    [VectorType(10)]
    public float[] Features { get; set; }
}

private static List<DataPoint> GenerateRandomDataPoints(int count,
    int featureVectorLenght)
{
    var examples = new List<DataPoint>();
    var rnd = new Random(0);
    for (int i = 0; i < count; ++i)
    {
        var example = new DataPoint();
        example.Features = new float[featureVectorLenght];
        var res = i % 4;
        // Generate random float feature values.
        for (int j = 0; j < featureVectorLenght; ++j)
        {
            var value = (float)rnd.NextDouble() + res * 0.2f;
            example.Features[j] = value;
        }

        // Generate label based on feature sum.
        if (res == 0)
            example.Label = "AA";
        else if (res == 1)
            example.Label = "BB";
        else if (res == 2)
            example.Label = "CC";
        else
            example.Label = "DD";
        examples.Add(example);
    }
    return examples;
}

var examples = GenerateRandomDataPoints(1000, 10);
var dataView = mlContext.Data.LoadFromEnumerable(examples);

In [ ]:
var pipeline = mlContext.Transforms.Conversion.MapValueToKey("Label")
                .Append(mlContext.MulticlassClassification.Trainers.SdcaMaximumEntropy());
var dataWithPredictions = pipeline.Fit(dataView).Transform(dataView);

In [ ]:
var newPipeline = mlContext.Transforms.Conversion.MapKeyToValue(new[]
        {
            new InputOutputColumnPair("LabelOriginalValue","Label"),
            new InputOutputColumnPair("PredictedLabelOriginalValue",
            "PredictedLabel")

        });
var transformedData = newPipeline.Fit(dataWithPredictions).Transform(dataWithPredictions);

In [ ]:
private class TransformedData
{
    public uint Label { get; set; }
    public uint PredictedLabel { get; set; }
    public string LabelOriginalValue { get; set; }
    public string PredictedLabelOriginalValue { get; set; }
}

transformedData = mlContext.Data.TakeRows(transformedData, 5);
var values = mlContext.Data.CreateEnumerable<TransformedData>(transformedData, reuseRowObject: false);
foreach (var row in values)
    Console.WriteLine($"{row.Label}\t\t{row.LabelOriginalValue}\t\t\t{row.PredictedLabel}\t\t\t{row.PredictedLabelOriginalValue}");